In [2]:
import pandas as pd
from pandas import read_csv
import numpy as np
import time
import pickle
from sklearn.model_selection import train_test_split

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.utils import np_utils
#from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error

from google.colab import drive
drive.mount('/content/drive')

input_dim = 13 # 4 if only scanners, 8 if scanners + distances

Mounted at /content/drive


In [6]:
def compare_params(old_params, new_params):
  return abs(old_params['error'] - new_params['error']) <= 0.10

def train_and_evaluate(X_train, Y_train, X_test, Y_test, X_val, Y_val, params):
  time_start = time.time()
  model = Sequential([
                      Dense(params['input_layer'], input_dim=input_dim, activation='sigmoid'),
                      BatchNormalization(),
                      Dense(params['first_layer'], activation='relu'),
                      Dense(params['second_layer'], activation='relu'),
                      Dense(2, activation='relu'),
  ])
  model.compile(optimizer='adam', loss='mse')
  history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=1000, batch_size=params['batch_size'], verbose=0)
  preds = model.predict(X_test)
  error = mean_squared_error(Y_test, preds)*3.048
  exec_time = time.time() - time_start
  return preds, error, exec_time

In [7]:
from sklearn.utils import shuffle

def load_data():
  with open('/content/drive/MyDrive/ColabNotebooks/rssi_labeled.csv') as f:
    rssi_values = read_csv(f, index_col=0)
    #rssi_values = read_csv(f)
  rssi_values.drop(['date'], axis=1, inplace=True)
  return rssi_values

def split_in_n_parts(data, n=10):
  return np.array_split(data, n)

def filter_fun(x):
  return len(x) != 0

def cross_validate(split, params):
  error_sum = 0
  results_list = []
  for i in range(len(split)):
    X_train = split.copy()

    X_test = X_train[i].copy()
    X_val = X_train[(i+1)%len(split)].copy()
    X_train[i] = pd.DataFrame()
    X_train[(i+1)%len(split)] = pd.DataFrame()
    X_train = list(filter(filter_fun, X_train))
    X_train = pd.concat(X_train)

    # separar o treino
    Y_train = X_train[['x', 'y']]
    X_train.drop(['x', 'y'], axis=1, inplace=True)
    # separar o teste
    Y_test = X_test[['x', 'y']]
    X_test.drop(['x', 'y'], axis=1, inplace=True)
    # separar o val
    Y_val = X_val[['x', 'y']]
    X_val.drop(['x', 'y'], axis=1, inplace=True)

    # preds, error, exec_time
    preds, error, exec_time = train_and_evaluate(X_train, Y_train, X_test, Y_test, X_val, Y_val, params)
    results_list += [(preds, error, exec_time)]
    error_sum += error
  return error_sum/len(split), results_list
  
def save_fstlay_results(input, fst_lay, result):
  with open(f'/content/drive/MyDrive/ColabNotebooks/results-kag-{input}-{fst_lay}.pkl', 'wb') as f:
    pickle.dump(result, f)

In [9]:
#################### GRID SEARCH ####################
rssi_values = load_data()
random_rssi_values = shuffle(rssi_values, random_state=7)
splitted_rssi_values = split_in_n_parts(random_rssi_values)

result = []
neurons_per_layer = [100]#[y for y in range(10, 101, 10)]
neurons_fst_layer = [60]#[y for y in range(10, 101, 10)]
input_neurons = [26]# [3,6] # [8] if only scanners, [12] if scanners + dists
# 'first_layer': 90, 
for inp in input_neurons:
  for fst_lay in neurons_fst_layer:
    for snd_lay in neurons_per_layer:
      params = {'input_layer': inp, 'first_layer': fst_lay, 'second_layer': snd_lay, 'batch_size': 300}
      print(f'Predicting current values: {params}')
      error, results_list = cross_validate(splitted_rssi_values, params)
      #_, error, model, exec_time = train_and_evaluate(X_train, Y_train, X_test, Y_test, X_val, Y_val, params)
      result += [{
          'params': params,
          'error': error,
          'results_list': results_list
      }]
    #save_fstlay_results(inp, fst_lay, result)  
    print(result)

Predicting current values: {'input_layer': 26, 'first_layer': 60, 'second_layer': 100, 'batch_size': 300}
[{'params': {'input_layer': 26, 'first_layer': 60, 'second_layer': 100, 'batch_size': 300}, 'error': 42.40977970071285, 'results_list': [(array([[10.38143  ,  3.4013689],
       [11.05866  ,  3.6508071],
       [10.11847  ,  3.3145833],
       [18.711203 ,  3.2206163],
       [13.833415 ,  4.450277 ],
       [ 4.1458583, 14.825072 ],
       [13.671291 ,  3.7341058],
       [13.490848 ,  3.7923748],
       [ 9.726637 ,  5.475967 ],
       [13.722771 ,  3.6970499],
       [22.197752 , 15.166997 ],
       [18.868916 ,  3.8615801],
       [12.330936 ,  6.412968 ],
       [13.803851 ,  3.6832855],
       [10.413513 ,  3.4060173],
       [18.718746 ,  2.9832418],
       [14.533137 ,  3.0997944],
       [10.082431 ,  3.4713156],
       [14.417551 ,  6.2156568],
       [18.99025  ,  4.235352 ],
       [10.082431 ,  3.4713156],
       [10.948508 ,  6.17782  ],
       [18.76331  ,  3.4571676